In [1]:
from datetime import date
from collections import defaultdict
from nltk import word_tokenize
from nltk.stem.porter import *
from nltk.util import ngrams
import string
import re

In [6]:
stopword_set = set()
file = open("~/stop.txt", 'r')
for line in file.readlines():
    word = line[:line.find('|')] if '|' in line else line
    word = word.strip()
    if len(word) > 0:
        stopword_set.add(word)
stopword_set

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 "can't",
 'cannot',
 'could',
 "couldn't",
 'did',
 "didn't",
 'do',
 'does',
 "doesn't",
 'doing',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 "hadn't",
 'has',
 "hasn't",
 'have',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 "he's",
 'her',
 'here',
 "here's",
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 "how's",
 'i',
 "i'd",
 "i'll",
 "i'm",
 "i've",
 'if',
 'in',
 'into',
 'is',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 "let's",
 'me',
 'more',
 'most',
 "mustn't",
 'my',
 'myself',
 'no',
 'nor',
 'not',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'ought',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'same',
 "shan't",
 'she',
 "she'd",
 "she'll",
 "she's",
 'should',
 "s

In [8]:
file_names = {}
file_names['speakermap'] = [str(i) + "_SpeakerMap.txt" for i in range(106, 115)]
file_names['descr'] = ["descr_" + str(i) + ".txt" for i in range(106, 115)]
file_names['speeches'] = ["speeches_" + str(i) + ".txt" for i in range(106, 115)]
file_names

{'speakermap': ['106_SpeakerMap.txt',
  '107_SpeakerMap.txt',
  '108_SpeakerMap.txt',
  '109_SpeakerMap.txt',
  '110_SpeakerMap.txt',
  '111_SpeakerMap.txt',
  '112_SpeakerMap.txt',
  '113_SpeakerMap.txt',
  '114_SpeakerMap.txt'],
 'descr': ['descr_106.txt',
  'descr_107.txt',
  'descr_108.txt',
  'descr_109.txt',
  'descr_110.txt',
  'descr_111.txt',
  'descr_112.txt',
  'descr_113.txt',
  'descr_114.txt'],
 'speeches': ['speeches_106.txt',
  'speeches_107.txt',
  'speeches_108.txt',
  'speeches_109.txt',
  'speeches_110.txt',
  'speeches_111.txt',
  'speeches_112.txt',
  'speeches_113.txt',
  'speeches_114.txt']}

In [9]:
speeches = {}
for file_name in file_names['speakermap']:
    print(file_name)
    file = open("hein-daily/" + file_name, 'r')
    next(file)
    for line in file.readlines():
        arr = line.split("|")
        speechID = arr[1]
        party = arr[7]
        if party in ['D', 'R']:
            speeches[speechID] = {'party': party}
    file.close()
speeches

106_SpeakerMap.txt
107_SpeakerMap.txt
108_SpeakerMap.txt
109_SpeakerMap.txt
110_SpeakerMap.txt
111_SpeakerMap.txt
112_SpeakerMap.txt
113_SpeakerMap.txt
114_SpeakerMap.txt


{'1060000007': {'party': 'D'},
 '1060000016': {'party': 'R'},
 '1060000023': {'party': 'R'},
 '1060000024': {'party': 'R'},
 '1060000027': {'party': 'R'},
 '1060000030': {'party': 'R'},
 '1060000032': {'party': 'D'},
 '1060000034': {'party': 'D'},
 '1060000036': {'party': 'D'},
 '1060000040': {'party': 'R'},
 '1060000042': {'party': 'R'},
 '1060000044': {'party': 'R'},
 '1060000045': {'party': 'R'},
 '1060000048': {'party': 'R'},
 '1060000050': {'party': 'R'},
 '1060000052': {'party': 'R'},
 '1060000054': {'party': 'R'},
 '1060000056': {'party': 'R'},
 '1060000058': {'party': 'R'},
 '1060000060': {'party': 'R'},
 '1060000062': {'party': 'R'},
 '1060000064': {'party': 'R'},
 '1060000066': {'party': 'R'},
 '1060000068': {'party': 'R'},
 '1060000070': {'party': 'R'},
 '1060000072': {'party': 'R'},
 '1060000074': {'party': 'R'},
 '1060000076': {'party': 'R'},
 '1060000077': {'party': 'R'},
 '1060000079': {'party': 'R'},
 '1060000081': {'party': 'R'},
 '1060000083': {'party': 'D'},
 '106000

In [ ]:
for filename in file_names['descr']:
    print(filename)
    file = open("~/hein-daily/" + filename, 'r')
    next(file)
    for line in file.readlines():
        arr = line.split("|")
        speech_id = arr[0]
        if speech_id in speeches:
            date_string = arr[2]
            year = int(date_string[:4])
            if year >= 2000:
                month = int(date_string[4:6])
                day = int(date_string[6:])
                speeches[speech_id]['date'] = date(year, month, day)
            else:
                speeches.pop(speech_id)
    file.close()
speeches

In [ ]:
stopword_set = set(stopwords.words('english'))
# stemmer = PorterStemmer()
bigrams = defaultdict(set)
for filename in file_names['speeches']:
    print(filename)
    file = open("~/hein-daily/" + filename, 'rb')
    next(file)
    for line in file.readlines():
        arr = line.decode("ascii", "ignore").split("|")
        speech_id = arr[0]
        if speech_id in speeches:
            text = arr[1].rstrip().lower()
            text = re.sub('[' + string.punctuation + ']', '', text) # remove punctuation
            tokens = word_tokenize(text)
            # tokens = [str(stemmer.stem(word)) for word in tokens if word not in stopwordSet]
            tokens = [word for word in tokens if word not in stopword_set]
            bigram_set = set(ngrams(tokens,2))
            for bigram in bigram_set:
                bigrams[bigram].add(speech_id)
    file.close()
bigrams